## Heuristik

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb

Die Funktion `heuristic(s, p)` dient zu einer hilfsweisen Berechnung von Siegwahrscheinlichkeiten anhand der Gegebenheiten. 
Je höher der Wert ist, umso besser stehen die Siegwahrscheinlichkeiten von `p`.

Vorhandene Steine zählen einfach in die Berechnung, vorhandene Mühlen vierfach und mögliche Mühlen doppelt.

Der schlechteste mögliche Wert ist -0.9, der beste 0.9.

In [ ]:
def heuristic(s, p):
    op = opponent(p)
    (_, board) = s
    # finding possible mills in next states
    pPossMills = sum(len(findMills(nB, p)) for (_, nB) in nextStates(s, p))
    opPossMills = sum(len(findMills(nB, op)) for (_, nB) in nextStates(s, op))
    # adding all score parameters
    pScore = countStones(s, p) + len(findMills(board, p)) * 4 + pPossMills * 2
    opScore = countStones(s, op) + len(findMills(board, op)) * 4 + opPossMills * 2
    # subtracting opponent score from own players score
    result = pScore - opScore
    # limeting the score to not overwrite real value score
    if result > 90:
        result = 90
    if result < -90:
        result = -90
    return result / 100